In [122]:
#Importando librerias necesarias
from datetime import datetime
from bs4 import BeautifulSoup 
import xlsxwriter as xl
import requests
import csv

In [123]:
#Funcion de llenar una pagina datos
def FillData(articles):
    #data = [['Oferta','URL','Empresa','Calificación','Ubicación','FechaBusqueda','TiempoPublicación']]
    dataout = []
    for article in articles:
            try: workOffer = article.find('h1',class_="fs18 fwB").text
            except: workOffer = ""
            try: workUrl = "https://www.computrabajo.com.pe/"+article.select('a[href]')[0]['href']
            except: workUrl = ""
            try: workBusiness = article.find('a',class_="fc_base hover it-blank").text
            except: workBusiness = ""
            try: workStart = float(article.find('span',class_="ml10 mr10").text.split()[1].replace(",","."))
            except: workStart = 0
            try:
                txt = article.find('p', class_="fs16 fc_base mt5 mb10")
                workLocation = txt.text.split()[-2]+" "+ txt.text.split()[-1]
            except : workLocation = ""
            try :
                workDateSeach = datetime.today().strftime('%d/%m/%Y')
                workTime = article.find('p',class_="fs13 fc_aux").text
            except: 
                workDateSeach = ""
                workTime = ""
            dataout.append([workOffer,workUrl,workBusiness,workStart,workLocation,workDateSeach,workTime])
    #print(f"tamDataPagina: {len(dataout)}")
    return dataout

In [124]:
#Funcion llenar todas
def SeachAll(NombreTrabajo,LugarTrabajo,NumeroPagsBusqueda):
    Data = [['Oferta','URL','Empresa','Calificación','Ubicación','FechaBusqueda','TiempoPublicación']]
    #print(f"Numero de SeachAll: {NumeroPagsBusqueda}")
    for i in range(NumeroPagsBusqueda):
        print(f"Busqueda en pag: {i+1}")
        page = requests.get(f"https://www.computrabajo.com.pe/trabajo-de-{NombreTrabajo}-en-{LugarTrabajo}?by=publicationtime&p={i+1}")
        doc = BeautifulSoup(page.content,'html.parser')
        articles = doc.find_all('article')
        #print(f"tamaño articles: {len(articles)}")
        #Data = Data + FillData(articles)
        Data.extend(FillData(articles))
        #print(f"tamaño DATA: {len(Data)}")
        articles = []
    return Data

In [125]:
#Funcion Guardar CSV
def Guardar_csv(nombre,data):
    with open(f'{nombre}.csv', 'w', encoding='utf-8',newline = '') as file:
        writer = csv.writer(file)
        for dat in data:
            writer.writerow(dat)
    print("\nDatos CSV Guardados!")

In [126]:
#Funcion Guardar xlsx
def Guardar_xlsx(nombre, datos):
    archivo = xl.Workbook(f'{nombre}.xlsx')
    hoja,tam = archivo.add_worksheet(), len(datos)
    for j in range(tam):
        for i in range(len(datos[j])):
            hoja.write(j,i,datos[j][i])
    archivo.close()
    print("\nDatos EXCEL Guardados!")
    

In [137]:
#Programa Principal!
##############################
NombreTrabajo = input("Escriba el trabajo que busca: ").lower().replace(" ","-")
while not NombreTrabajo : NombreTrabajo = input("Escriba UN TRABAJO!\nIngrese trabajo otra vez: ").lower().replace(" ","-")
LugarTrabajo = input("Escriba el lugar de trabajo: ").lower().replace(" ","-")
#url = f"https://www.computrabajo.com.pe/trabajo-de-{NombreTrabajo}-en-{LugarTrabajo}?by=publicationtime&p={NumeroPagsBusqueda}"
url = f"https://www.computrabajo.com.pe/trabajo-de-{NombreTrabajo}-en-{LugarTrabajo}?by=publicationtime"
page = requests.get(url)
data,nombre = [], ""
#Generar los datos
try:
    document = BeautifulSoup(page.content,'html.parser')
    try : 
        numero = int(document.find('span', class_="fwB mr5").text.split()[0])
        if numero > 0 : print(f"\nSe encontro {numero} trabajos!")
        NumeroPagsBusqueda = (numero // 20) + 1
        print(f"En {NumeroPagsBusqueda} Paginas de Computrabajo!")
        #NumeroPagsBusqueda = int(input("Escriba cuantas paginas quiere buscar: "))
        #while NumeroPagsBusqueda < 0 : NumeroPagsBusqueda = int(input("Numero de paginas debe ser POSITIVO!\nEscriba cuantas paginas de nuevo: "))
        data = SeachAll(NombreTrabajo,LugarTrabajo,NumeroPagsBusqueda)
        #print(data)
        #Guardando Datos
        nombre = input("Ingrese que nombre tendran los archivos : ")
        Guardar_csv(nombre,data)
        Guardar_xlsx(nombre, data)
    except : print("Algo a fallado, o no se a encontrado :(")
except requests.exceptions.HTTPError as e:  
    print("Error: " + str(e)+"\nError en la Busqueda:(")


Escriba el trabajo que busca: data
Escriba el lugar de trabajo: callao

Se encontro 28 trabajos!
En 2 Paginas de Computrabajo!
Busqueda en pag: 1
Busqueda en pag: 2
Ingrese que nombre tendran los archivos : jobs_in_data

Datos CSV Guardados!

Datos EXCEL Guardados!
